## Part 2 - Linear Regression
Examining the relationship between a player's pass volume and completion percentage

In [5]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

Load data from StatsBomb GitHub page.

In [6]:
base_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/"
comp_url = base_url + "matches/{}/{}.json"
match_url = base_url + "events/{}.json"

We will use a modification of the `parse_data` function were we extract `passes` and only the attributes `player_id` and `outcome`.

In [7]:
def parse_data(competition_id, season_id):
    matches = requests.get(url=comp_url.format(competition_id, season_id)).json()
    match_ids = [match['match_id'] for match in matches]

    all_events = []
    for match_id in match_ids:

        events = requests.get(url=match_url.format(match_id)).json()
        
        passes = [x for x in events if x['type']['name'] == "Pass"]
        for p in passes:
            attributes = {
                "player_id": p['player']['id'],
                "outcome": 0 if 'outcome' in p['pass'].keys() else 1,
            }
            all_events.append(attributes)
            
    return pd.DataFrame(all_events)

In [ ]:
WC_id = 43
season_id = 3
df = parse_data(WC_id, season_id)

In [ ]:
from sklearn.Linear_Model import LinearRegression